# Packages

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
from keras.preprocessing import image

# Configuration

In [ ]:
SHAPE = 80
EPOCHS = 5 
NUM_CLASSES = 7

MODEL_FOLDER = "./data/trained_models/"

# Data generation

In [ ]:
# Configure the training image generator
TRAINING_DIR = "./data/train/"
training_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

# Configure the validation image generator
VALIDATION_DIR = "./data/test/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

# Set the image size and batch size for training and validation
# generators 
train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(SHAPE,SHAPE),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(SHAPE,SHAPE),
    class_mode='categorical',
    batch_size=64
)

# Image informations

In [ ]:
class_names = list(validation_generator.class_indices.keys())

print(class_names)

# Model creation

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image SHAPExSHAPE with
    # 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(SHAPE, SHAPE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128 , (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation=tf.nn.softmax)
])

model.summary()

# Training

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_generator, epochs=EPOCHS, steps_per_epoch=100, validation_data = validation_generator, verbose = 1, validation_steps=3)

model.save(MODEL_FOLDER + "emotion.h5")

# History

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

# Validation

In [ ]:
# 'anger' - 0, 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise'

model = tf.keras.models.load_model(MODEL_FOLDER + "emotion.h5")

# paths of the pictures loaded from the uploaded folder
uploaded = []
# labels bases on the folder name
labels = []

for class_name in class_names:
    path = os.path.join('./data/uploaded/', class_name)
    for img in os.listdir(path):
        if (img == '.gitkeep'):
            continue
        uploaded.append(os.path.join(path, img))
        labels.append(class_name)


for i in range(len(labels)):
    labels[i] = class_names.index(labels[i])

print(uploaded)
print(labels)

images_predict = []
images = []

for img in uploaded:
    new_image = tf.keras.utils.load_img(img, target_size=(SHAPE, SHAPE))
    new_image_batch = tf.expand_dims(tf.keras.utils.img_to_array(new_image), 0)
    images_predict.append(new_image_batch)
    images.append(new_image)


In [ ]:
predictions_self = model.predict(images_predict[6])
print(predictions_self[0])

score = predictions_self[0]
# score = tf.nn.softmax(predictions_self[0])


print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
probability_model = tf.keras.Sequential([model])

predictions = probability_model.predict(images_predict[0])

print(predictions[0])
print(np.argmax(predictions[0]))

In [ ]:
def plot_image(predictions_array, true_label, img):
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label],
                                color=color)
                                )

def plot_value_array(predictions_array, true_label):
  plt.grid(False)
  plt.xticks(range(7))
  plt.yticks([])
  thisplot = plt.bar(range(7), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(predictions[i], labels[i], images[i])
plt.subplot(1,2,2)
plot_value_array(predictions[i],  labels[i])
plt.show()

## Image validation matrix
Here the uploaded images will be drawn in a matrix.


In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = len(images)
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  predictions = probability_model.predict(images_predict[i])
  print(predictions[0])
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(predictions[0], labels[i], images[i])
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(predictions[0], labels[i])
plt.tight_layout()
plt.show()